In [2]:
import os
import pandas as pd

# Define file paths
data_dir = "/Users/aakashsuresh/fairness/processed_data_nhanes_lab/"
files_to_load = [
    "fasting_questionnaire_processed.csv",
    "fasting_glucose_processed.csv",
    "glycohemoglobin_processed.csv",
    "biochemistry_profile_processed.csv",
    "iron_status_processed.csv",
    "c_reactive_protein_processed.csv",
    "cotinine_processed.csv",
]

# Load dataframes into a dictionary
dataframes = {}
for file in files_to_load:
    file_path = os.path.join(data_dir, file)
    df_name = file.replace("_processed.csv", "")
    dataframes[df_name] = pd.read_csv(file_path)

# Merge datasets on SEQN (NHANES unique identifier)
merged_df = dataframes["fasting_questionnaire"]
for name, df in dataframes.items():
    if name != "fasting_questionnaire":
        merged_df = pd.merge(merged_df, df, on="seqn", how="inner")

# Standardize column names to lowercase
df.columns = df.columns.str.strip().str.lower()

print("Data merged successfully. Shape:", merged_df.shape)


Data merged successfully. Shape: (0, 77)


In [3]:
# Remove fasting_questionnaire temporarily because it has no unique counts 
del dataframes["fasting_questionnaire"]

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Merge remaining datasets on 'seqn' (ignoring fasting_questionnaire)
merged_df = None

# Iterate and merge all datasets
for name, df in dataframes.items():
    if merged_df is None:
        merged_df = df  # Initialize with the first dataset
    else:
        merged_df = pd.merge(merged_df, df, on="seqn", how="inner")

print(f"Data merged successfully. Shape: {merged_df.shape}")

# Drop rows with missing values after merge
merged_df = merged_df.dropna()

# Standardize numeric columns
scaler = StandardScaler()
numeric_cols = merged_df.select_dtypes(include=["float64", "int64"]).columns
merged_df[numeric_cols] = scaler.fit_transform(merged_df[numeric_cols])

print("Data cleaned and scaled.")

Data merged successfully. Shape: (4526, 59)
Data cleaned and scaled.


In [5]:
# Display all column names in merged_df
print("Columns in merged_df:")
print(merged_df.columns.tolist())

Columns in merged_df:
['seqn', 'wtsafprp', 'lbxglu', 'lbdglusi', 'lbxgh', 'lbxsatsi', 'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si', 'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi', 'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc', 'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi', 'lbxsksi', 'lbxsnasi', 'lbxstb', 'lbdstbsi', 'lbdstblc', 'lbxsca', 'lbdscasi', 'lbxsch', 'lbdschsi', 'lbxstp', 'lbdstpsi', 'lbxstr', 'lbdstrsi', 'lbxsua', 'lbdsuasi', 'lbxirn', 'lbdirnsi', 'lbxuib', 'lbduiblc', 'lbduibsi', 'lbdtib', 'lbdtibsi', 'lbdpct', 'lbxhscrp', 'lbdhrplc', 'lbxcot', 'lbdcotlc', 'lbxhcot', 'lbdhcolc']


In [12]:
print("Columns in merged_df:", merged_df.columns)
print("Columns in demo_selected:", demo_selected.columns)

Columns in merged_df: Index(['seqn', 'wtsafprp', 'lbxglu', 'lbdglusi', 'lbxgh', 'lbxsatsi',
       'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si',
       'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi',
       'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc',
       'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi',
       'lbxsksi', 'lbxsnasi', 'lbxstb', 'lbdstbsi', 'lbdstblc', 'lbxsca',
       'lbdscasi', 'lbxsch', 'lbdschsi', 'lbxstp', 'lbdstpsi', 'lbxstr',
       'lbdstrsi', 'lbxsua', 'lbdsuasi', 'lbxirn', 'lbdirnsi', 'lbxuib',
       'lbduiblc', 'lbduibsi', 'lbdtib', 'lbdtibsi', 'lbdpct', 'lbxhscrp',
       'lbdhrplc', 'lbxcot', 'lbdcotlc', 'lbxhcot', 'lbdhcolc'],
      dtype='object')
Columns in demo_selected: Index(['SEQN', 'age', 'gender'], dtype='object')


In [15]:
print(demo_selected.columns)
print(merged_df.head())

Index(['seqn', 'age', 'gender'], dtype='object')
       seqn  wtsafprp    lbxglu  lbdglusi     lbxgh  lbxsatsi  lbdsatlc  \
0 -1.766579 -0.442652 -0.395739 -0.397905 -0.445597 -0.679824       0.0   
1 -1.765016 -0.578612 -0.229418 -0.228153 -0.173587 -0.679824       0.0   
2 -1.764346 -0.604835  1.184315  1.184782 -0.082917 -0.121378       0.0   
3 -1.763676 -0.374949 -0.534341 -0.532708 -0.445597 -0.527521       0.0   
4 -1.762559  0.299318 -0.451180 -0.452825 -0.264257 -0.222914       0.0   

     lbxsal  lbdsalsi  lbxsapsi  ...  lbxhscrp  lbdhrplc    lbxcot  lbdcotlc  \
0  0.754519  0.754519  1.679067  ... -0.498703  7.601264 -0.413056 -0.705116   
1 -0.685539 -0.685539 -0.004073  ...  3.228526 -0.131557  2.609282 -0.705116   
2 -0.109516 -0.109516 -0.785531  ... -0.304318 -0.131557 -0.413048 -0.705116   
3  0.754519  0.754519  1.298357  ... -0.192123 -0.131557 -0.413116  1.418207   
4  0.754519  0.754519 -0.284596  ... -0.107324 -0.131557 -0.412777 -0.705116   

    lbxhcot  lbdhco

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Load demographic data
demo_df = pd.read_sas("P_DEMO.xpt", format="xport")

# Select relevant demographic features that exist
demo_selected = demo_df[["SEQN", "RIDAGEYR", "RIAGENDR"]]

# Convert column names to lowercase
merged_df.columns = merged_df.columns.str.lower()
demo_selected.columns = demo_selected.columns.str.lower()

# Merge again
merged_df = merged_df.merge(demo_selected, on="seqn", how="left")

# Select final features
selected_features = ["RIDAGEYR", "RIAGENDR", "lbxglu", "lbxgh"]  # Cholesterol & Triglycerides

# Ensure only selected features are used
X = merged_df[selected_features]
y = merged_df[["lbxglu", "lbxgh"]]  # Multi-output target

# Drop rows with missing values
X = X.dropna()
y = y.loc[X.index]  # Align target with feature rows

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
base_model = RandomForestRegressor(n_estimators=100, random_state=42)
model = MultiOutputRegressor(base_model)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate mean squared error
glu_mse = mean_squared_error(y_test["lbxglu"], y_pred[:, 0])
gh_mse = mean_squared_error(y_test["lbxgh"], y_pred[:, 1])

print(f"Fasting Glucose (lbxglu) MSE: {glu_mse:.4f}")
print(f"Glycohemoglobin (lbxgh) MSE: {gh_mse:.4f}")

KeyError: "['RIDAGEYR', 'RIAGENDR'] not in index"

In [36]:
# Load demographic data
demo_df = pd.read_sas("P_DEMO.xpt", format="xport")

# Select relevant demographic features that exist
demo_selected = demo_df[["SEQN", "RIDAGEYR", "RIAGENDR"]]

# Convert column names to lowercase for both merged_df and demo_selected
merged_df.columns = merged_df.columns.str.lower()
demo_selected.columns = demo_selected.columns.str.lower()

# Check columns of merged_df before merging
print(f"merged_df columns before merge: {merged_df.columns}")
print(f"demo_selected columns before merge: {demo_selected.columns}")

# Merge again with custom suffixes to avoid conflicts
merged_df = merged_df.merge(demo_selected, on="seqn", how="left", suffixes=('_orig', '_demo'))

# Check columns of merged_df after merge
print(f"merged_df columns after merge: {merged_df.columns}")

# Select final features
selected_features = ["ridageyr", "riagendr", "lbxglu", "lbxgh"]  # Cholesterol & Triglycerides

# Ensure only selected features are used
X = merged_df[selected_features]
y = merged_df[["lbxglu", "lbxgh"]]  # Multi-output target

# Drop rows with missing values
X = X.dropna()
y = y.loc[X.index]  # Align target with feature rows

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
base_model = RandomForestRegressor(n_estimators=100, random_state=42)
model = MultiOutputRegressor(base_model)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate mean squared error
glu_mse = mean_squared_error(y_test["lbxglu"], y_pred[:, 0])
gh_mse = mean_squared_error(y_test["lbxgh"], y_pred[:, 1])

print(f"Fasting Glucose (lbxglu) MSE: {glu_mse:.4f}")
print(f"Glycohemoglobin (lbxgh) MSE: {gh_mse:.4f}")

merged_df columns before merge: Index(['seqn', 'wtsafprp', 'lbxglu', 'lbdglusi', 'lbxgh', 'lbxsatsi',
       'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si',
       'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi',
       'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc',
       'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi',
       'lbxsksi', 'lbxsnasi', 'lbxstb', 'lbdstbsi', 'lbdstblc', 'lbxsca',
       'lbdscasi', 'lbxsch', 'lbdschsi', 'lbxstp', 'lbdstpsi', 'lbxstr',
       'lbdstrsi', 'lbxsua', 'lbdsuasi', 'lbxirn', 'lbdirnsi', 'lbxuib',
       'lbduiblc', 'lbduibsi', 'lbdtib', 'lbdtibsi', 'lbdpct', 'lbxhscrp',
       'lbdhrplc', 'lbxcot', 'lbdcotlc', 'lbxhcot', 'lbdhcolc', 'age_x',
       'gender_x', 'riagendr_x', 'ridageyr_x', 'age_y', 'gender_y',
       'ridageyr_y', 'riagendr_y', 'ridageyr', 'riagendr'],
      dtype='object')
demo_selected columns before merge: Index(['seqn', 'ridageyr', 'riagendr'], dtyp

KeyError: "['ridageyr', 'riagendr'] not in index"

In [37]:
# After merging, check the columns to ensure correct names
print(f"merged_df columns after merge: {merged_df.columns}")

# Then, try selecting the columns again using the exact column names
selected_features = ["ridageyr_demo", "riagendr_demo", "lbxglu", "lbxgh"]  # Adjusted column names based on suffixes

# Ensure only selected features are used
X = merged_df[selected_features]
y = merged_df[["lbxglu", "lbxgh"]]  # Multi-output target

# Drop rows with missing values
X = X.dropna()
y = y.dropna()

# Check the shapes
print(f"X shape: {X.shape}, y shape: {y.shape}")


merged_df columns after merge: Index(['seqn', 'wtsafprp', 'lbxglu', 'lbdglusi', 'lbxgh', 'lbxsatsi',
       'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si',
       'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi',
       'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc',
       'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi',
       'lbxsksi', 'lbxsnasi', 'lbxstb', 'lbdstbsi', 'lbdstblc', 'lbxsca',
       'lbdscasi', 'lbxsch', 'lbdschsi', 'lbxstp', 'lbdstpsi', 'lbxstr',
       'lbdstrsi', 'lbxsua', 'lbdsuasi', 'lbxirn', 'lbdirnsi', 'lbxuib',
       'lbduiblc', 'lbduibsi', 'lbdtib', 'lbdtibsi', 'lbdpct', 'lbxhscrp',
       'lbdhrplc', 'lbxcot', 'lbdcotlc', 'lbxhcot', 'lbdhcolc', 'age_x',
       'gender_x', 'riagendr_x', 'ridageyr_x', 'age_y', 'gender_y',
       'ridageyr_y', 'riagendr_y', 'ridageyr_orig', 'riagendr_orig',
       'ridageyr_demo', 'riagendr_demo'],
      dtype='object')
X shape: (0, 4), y shape: (452

In [39]:
# Check for missing values in the selected columns
missing_values = merged_df[selected_features].isnull().sum()
print(f"Missing values in selected features:\n{missing_values}")


Missing values in selected features:
ridageyr_demo    4526
riagendr_demo    4526
lbxglu              0
lbxgh               0
dtype: int64


In [40]:
print(f"merged_df shape before selecting features: {merged_df.shape}")


merged_df shape before selecting features: (4526, 71)
